<a href="https://colab.research.google.com/github/beiyouwuyanzu/llama_application/blob/main/%E2%80%9CChinese_Vicuna_generate_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/Facico/Chinese-Vicuna

Cloning into 'Chinese-Vicuna'...
remote: Enumerating objects: 757, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 757 (delta 24), reused 23 (delta 14), pack-reused 713
Receiving objects: 100% (757/757), 257.93 MiB | 20.71 MiB/s, done.
Resolving deltas: 100% (395/395), done.


In [5]:
!pip install -r ./Chinese-Vicuna/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-e5zts2_6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-e5zts2_6
  Resolved https://github.com/huggingface/transformers.git to commit 137eb8e663ae948306ec22655cf57495a4493f33
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/peft (to revision e536616888d51b453ed354a6f1e243fecb02ea08) to /tmp/pip-req-build-xm86tjf9
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft /tmp/pip-req-build-xm86tjf9
  Running command git rev-parse -q --verify 'sha^e536616888d51b453ed354a6f1e243fecb02ea08'
  Running command git fetch -q https://github.com/huggingface/peft e536616888d51b453ed354a6f1e

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import sys
import torch
from peft import PeftModel, PeftModelForCausalLM, LoraConfig
import transformers
import gradio as gr
import argparse
import warnings
import os
import sys
sys.path.insert(0, "./Chinese-Vicuna")
from utils import SteamGenerationMixin

In [8]:
lora_path = "/content/drive/MyDrive/llama_data/checkpoint-200"

lora_bin_path = os.path.join(lora_path, "adapter_model.bin")
print(lora_bin_path)
if not os.path.exists(lora_bin_path) and 1:
    pytorch_bin_path = os.path.join(lora_path, "pytorch_model.bin")
    print(pytorch_bin_path)
    if os.path.exists(pytorch_bin_path):
        os.rename(pytorch_bin_path, lora_bin_path)
        warnings.warn(
            "The file name of the lora checkpoint'pytorch_model.bin' is replaced with 'adapter_model.bin'"
        )
    else:
        assert ('Checkpoint is not Found!')


/content/drive/MyDrive/llama_data/checkpoint-200/adapter_model.bin


In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
BASE_MODEL = "decapoda-research/llama-7b-hf"

device = "cpu"

if device == "cuda":
    model = LlamaForCausalLM.from_pretrained(
        BASE_MODEL,
        load_in_8bit=LOAD_8BIT,
        torch_dtype=torch.float16,
        device_map={"": 0},
    )
    model = SteamGenerationMixin.from_pretrained(
        model, LORA_WEIGHTS, torch_dtype=torch.float16, device_map={"": 0}
    )
elif device == "mps":
    model = LlamaForCausalLM.from_pretrained(
        BASE_MODEL,
        device_map={"": device},
        torch_dtype=torch.float16,
    )
    model = SteamGenerationMixin.from_pretrained(
        model,
        LORA_WEIGHTS,
        device_map={"": device},
        torch_dtype=torch.float16,
    )
else:
    model = LlamaForCausalLM.from_pretrained(
        BASE_MODEL, device_map={"": device}, low_cpu_mem_usage=True
    )
    model = SteamGenerationMixin.from_pretrained(
        model,
        LORA_WEIGHTS,
        device_map={"": device},
    )


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

如果你生成的网页信息被模型加载信息覆盖了，可以再跑一遍代码

If the information on your generated page is overwritten by the model loading information, you can run the code again

In [3]:
!python ./Chinese-Vicuna/generate.py --model_path decapoda-research/llama-7b-hf --lora_path "/content/drive/MyDrive/llama_data/checkpoint-200" --use_local 0

Traceback (most recent call last):
  File "/content/./Chinese-Vicuna/generate.py", line 3, in <module>
    from peft import PeftModel, PeftModelForCausalLM, LoraConfig
ModuleNotFoundError: No module named 'peft'


In [ ]:
!pip list | grep torch

torch                         1.13.1+cu116
torchaudio                    0.13.1+cu116
torchsummary                  1.5.1
torchtext                     0.14.1
torchvision                   0.14.1+cu116


In [ ]:
!whereis python

python: /usr/bin/python2.7 /usr/bin/python3.8 /usr/bin/python3.8-config /usr/bin/python3.9 /usr/bin/python3.9-config /usr/lib/python2.7 /usr/lib/python3.8 /usr/lib/python3.9 /etc/python2.7 /etc/python3.8 /etc/python3.9 /usr/local/bin/python /usr/local/lib/python3.9 /usr/local/lib/python2.7 /usr/local/lib/python3.8 /usr/include/python3.8 /usr/include/python3.9
